In [29]:
from kornia.augmentation import RandomJPEG
from compressai.zoo import load_state_dict
from Network import TestModel
from src.helpers.utils import load_model
from src.helpers.utils import logger_setup
from default_config import ModelModes
from torchvision import transforms,datasets
from torchvision.models import ResNet50_Weights
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
preproces_t = transforms.Compose([transforms.Resize((256,256)),transforms.CenterCrop((224,224)),transforms.ToTensor()]) 
testset = datasets.Imagenette(root='./data', split='val', download=False, transform=preproces_t)
indices = np.random.choice(len(testset), 100, replace=False)
testset = torch.utils.data.Subset(testset, indices)

In [30]:
def rescale(img: np.array) -> np.array:
        """Function to scale an image from float to uint8

        :param:img : np.array -- the image to rescale
        :return: rescaled image as numpy array
        """
        return (img * 255.9999).astype(np.uint8)

def jpeg_encode(image,path,q):
    image = image.squeeze().permute(1, 2, 0)
    im = Image.fromarray(rescale(image.numpy()), mode='RGB')
    im.save(path, "JPEG", quality=q)

In [31]:
import os
def bpp(path):
    file_size = os.path.getsize(path)
    return (file_size*8)/(224*224)

In [32]:
data_loader = torch.utils.data.DataLoader(testset, batch_size=1,
                                                 shuffle=False)

In [33]:
def test_jpeg(q):
    bpps = []
    for i,(image,target) in enumerate(data_loader):
        jpeg_encode(image,f'images/{i}.jpeg',q)
    for f in os.listdir('images/'):
        bpps.append(bpp('images/'+f))
    return sum(bpps)/len(bpps)
    
    
    

In [34]:
for q in [5,10,15,25,35,50,75,95]:
    print(f'BPP for {q}:{test_jpeg(q)}')

BPP for 5:0.3501480728429984
BPP for 10:0.4806684431198222
BPP for 15:0.5924160815316224
BPP for 25:0.7788489467569204
BPP for 35:0.9383445519296828
BPP for 50:1.138601232572237
BPP for 75:1.650194483734088
BPP for 95:3.7962877601535645


In [28]:
print(f'BPP for {1}:{test_jpeg(1)}')

BPP for 1:0.03517223334259446


In [81]:
class ELIC():
    def __init__(self,weights = '0016'):
        self.state_dict = load_state_dict(torch.load(f'data/ELIC_{weights}_ft_3980_Plateau.pth.tar'))
        self.model = TestModel().from_state_dict(self.state_dict)
        self.model = self.model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        self.model.eval()
    def compress(self,x):
        x = transforms.Resize((256,256))(x)
        return self.model.compress(x)['strings']
        
    
    

In [84]:
def test_ELIC(weights):
    elic_model = ELIC(weights = weights)
    bpps = []
    for i,(image,target) in enumerate(data_loader):
        strings = elic_model.compress(image)
        bpp = 0
        for s in strings:
            for j in s:
                if isinstance(j, list):
                    for i in j:
                        if isinstance(i, list):
                            for k in i:
                                bpp += len(k)
                        else:
                            bpp += len(i)
                else:
                    bpp += len(j)
        bpp *= 8.0 / (256*256)
        bpps.append(bpp)
    return sum(bpps)/100

In [86]:
for q in ['0004','0008','0016','0032','0150','0450']:
    print(f'BPP for {q}:{test_ELIC(q)}')

/tmp/ipykernel_1055095/911578149.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.state_dict = load_state_dict(torch.load(f'data/ELIC_{weights}_ft_3980_Plateau.pth.t

BPP for 0004:0.063408203125
BPP for 0008:0.093271484375
BPP for 0016:0.135673828125
BPP for 0032:0.19384765625
BPP for 0150:0.41533203125
BPP for 0450:0.6925927734375


In [5]:
class HiFiC():

    def __init__(self, weights='hific_low.pt',):

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.logger = logger_setup(logpath=os.path.join('data/logs/', 'logs'), filepath=os.path.abspath('data/logs'))
        self.args,self.transform_model,self.optimizers = load_model('data/'+ weights,self.logger,self.device,model_mode=ModelModes.EVALUATION)

    def compress(self, x):
        compression_output,attained_bpp = self.transform_model(x)
        #attained_bpp = 32 * ((len(compression_output.hyperlatents_encoded) + len(compression_output.latents_encoded)) / np.prod((224,224)))
        return attained_bpp

In [6]:
def test_HiFiC(weights):
    hific_model = HiFiC(weights = weights)
    bpps = []

    for i,(image,target) in enumerate(data_loader):
        with torch.no_grad():
            bpp = hific_model.compress(image.to('cuda'))
            bpps.append(bpp)
    return sum(bpps)/100

In [7]:
output=[]
for q in ['hific_low.pt','hific_med.pt','hific_hi.pt']:
    output.append(f'BPP for {q}:{test_HiFiC(q)}')

14:06:18 INFO - logger_setup: /home/jovyan/data/logs
INFO:src.helpers.utils:/home/jovyan/data/logs
/home/jovyan/src/helpers/utils.py:175: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

Building prior probability tables...


100%|█████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 1019.60it/s]

Setting up Perceptual loss...



/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jovyan/src/loss/perceptual_similarity/dist_model.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrus

...

Building prior probability tables...


100%|█████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 1015.26it/s]

Setting up Perceptual loss...



14:06:33 INFO - load_model: Loading model ...
14:06:33 INFO - load_model: Loading model ...
14:06:33 INFO - load_model: Loading model ...
INFO:src.helpers.utils:Loading model ...
14:06:33 INFO - load_model: MODEL TYPE: compression_gan
14:06:33 INFO - load_model: MODEL TYPE: compression_gan
14:06:33 INFO - load_model: MODEL TYPE: compression_gan
INFO:src.helpers.utils:MODEL TYPE: compression_gan
14:06:33 INFO - load_model: MODEL MODE: evaluation
14:06:33 INFO - load_model: MODEL MODE: evaluation
14:06:33 INFO - load_model: MODEL MODE: evaluation
INFO:src.helpers.utils:MODEL MODE: evaluation
14:06:33 INFO - load_model: Model(
  (Encoder): Encoder(
    (pre_pad): ReflectionPad2d((3, 3, 3, 3))
    (asymmetric_pad): ReflectionPad2d((0, 1, 1, 0))
    (post_pad): ReflectionPad2d((1, 1, 1, 1))
    (conv_block1): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 60, kernel_size=(7, 7), stride=(1, 1))
      (2): ChannelNorm2D()
      (3): ReLU()
    )
    (conv_block2): 

Loading model from: /home/jovyan/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [8]:
for o in output:
    print(o)

BPP for hific_low.pt:0.15450650453567505
BPP for hific_med.pt:0.42845335602760315
BPP for hific_hi.pt:0.46835437417030334
